Loading the dataset

In [51]:
!unzip /content/drive/MyDrive/dev-clean.zip -d /content/audio

Streaming output truncated to the last 5000 lines.
  inflating: /content/audio/dev-clean/5338/284437/5338_284437_000018_000001.normalized.txt  
  inflating: /content/audio/dev-clean/5338/284437/5338_284437_000018_000001.wav  
  inflating: /content/audio/dev-clean/5338/284437/5338_284437_000020_000000.normalized.txt  
  inflating: /content/audio/dev-clean/5338/284437/5338_284437_000020_000000.wav  
  inflating: /content/audio/dev-clean/5338/284437/5338_284437_000021_000000.normalized.txt  
  inflating: /content/audio/dev-clean/5338/284437/5338_284437_000021_000000.wav  
  inflating: /content/audio/dev-clean/5338/284437/5338_284437_000023_000000.normalized.txt  
  inflating: /content/audio/dev-clean/5338/284437/5338_284437_000023_000000.wav  
  inflating: /content/audio/dev-clean/5338/284437/5338_284437_000023_000001.normalized.txt  
  inflating: /content/audio/dev-clean/5338/284437/5338_284437_000023_000001.wav  
  inflating: /content/audio/dev-clean/5338/284437/5338_284437_000023_00000

In [53]:
path = '/content/audio'
print("/content/dev.zip", path)

/content/dev.zip /content/audio


In [54]:
import os

print("Files in dataset folder:")
print(os.listdir(path))


Files in dataset folder:
['dev-clean']


In [55]:
import os
import librosa

# Example audio file from the extracted dataset
audio_path = "/content/audio/dev-clean/6241/61946/6241_61946_000051_000001.wav"

# Load the audio
audio, sr = librosa.load(audio_path, sr=None)

print("Audio loaded successfully!")
print("Sample rate:", sr)
print("Duration (seconds):", librosa.get_duration(y=audio, sr=sr))


Audio loaded successfully!
Sample rate: 24000
Duration (seconds): 6.55


In [56]:
#To process all files
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".wav"):
            full_path = os.path.join(root, file)
            print("Processing:", full_path)
            # load/process audio here

Processing: /content/audio/dev-clean/1993/147965/1993_147965_000006_000000.wav
Processing: /content/audio/dev-clean/1993/147965/1993_147965_000006_000002.wav
Processing: /content/audio/dev-clean/1993/147965/1993_147965_000007_000000.wav
Processing: /content/audio/dev-clean/1993/147965/1993_147965_000007_000004.wav
Processing: /content/audio/dev-clean/1993/147965/1993_147965_000003_000003.wav
Processing: /content/audio/dev-clean/1993/147965/1993_147965_000001_000001.wav
Processing: /content/audio/dev-clean/1993/147965/1993_147965_000005_000001.wav
Processing: /content/audio/dev-clean/1993/147965/1993_147965_000002_000003.wav
Processing: /content/audio/dev-clean/1993/147965/1993_147965_000006_000001.wav
Processing: /content/audio/dev-clean/1993/147965/1993_147965_000001_000006.wav
Processing: /content/audio/dev-clean/1993/147965/1993_147965_000003_000000.wav
Processing: /content/audio/dev-clean/1993/147965/1993_147965_000002_000004.wav
Processing: /content/audio/dev-clean/1993/147965/199

## Implement audio transcription for a provided audio file

In [57]:
!pip install -U openai-whisper


In [59]:
import glob

# Get all .wav files recursively in your dataset
audio_files = glob.glob("/content/audio/dev-clean/**/*.wav", recursive=True)

print(f"Found {len(audio_files)} audio files.")


Found 5736 audio files.


In [84]:
from collections import Counter
import os
import glob

audio_files = glob.glob("/content/audio/dev-clean/**/*.wav", recursive=True)
folders = [os.path.basename(os.path.dirname(f)) for f in audio_files]
count = Counter(folders)

print(count)

Counter({'4944': 176, '121550': 153, '149220': 138, '66125': 125, '83699': 123, '66129': 122, '76958': 120, '24833': 119, '147961': 118, '83705': 116, '15664': 116, '170142': 116, '93302': 114, '137482': 108, '137823': 105, '142785': 104, '163249': 103, '149897': 102, '154328': 102, '275155': 100, '105575': 100, '4943': 96, '170145': 95, '110124': 95, '275154': 91, '64029': 86, '34615': 85, '170138': 84, '93306': 81, '9008': 80, '121123': 80, '149896': 78, '284437': 76, '118436': 75, '275156': 74, '61946': 71, '64038': 70, '136532': 68, '281318': 66, '34629': 62, '110523': 62, '141231': 61, '275224': 56, '147956': 56, '147960': 56, '168635': 56, '64726': 54, '286674': 53, '153954': 51, '61943': 50, '84280': 49, '50561': 45, '64257': 44, '34622': 43, '154320': 42, '161169': 42, '43358': 42, '8226': 41, '73752': 40, '64025': 39, '111771': 39, '147149': 37, '147966': 36, '148538': 36, '43359': 36, '152373': 35, '147965': 34, '149874': 33, '142845': 33, '143397': 30, '135031': 29, '43363':

In [88]:
import shutil
import os

folder_path = "/content/transcriptions_json"

# Check if folder exists
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)  # Delete the entire folder and its contents
    print(f"✅ Cleared: {folder_path}")
    os.makedirs(folder_path, exist_ok=True)  # Recreate empty folder
    print(f"📁 Recreated: {folder_path}")
else:
    print(f"⚠️ Folder not found: {folder_path}")


✅ Cleared: /content/transcriptions_json
📁 Recreated: /content/transcriptions_json


In [89]:
import random
from collections import defaultdict
import os
import glob
import json
import whisper

# Load Whisper model
model = whisper.load_model("small")

# Get all audio files
audio_files = glob.glob("/content/audio/dev-clean/**/*.wav", recursive=True)

# Group files by immediate parent folder
files_by_folder = defaultdict(list)
for f in audio_files:
    folder = os.path.basename(os.path.dirname(f))
    files_by_folder[folder].append(f)

# Select only 5 random folders
selected_folders = random.sample(list(files_by_folder.keys()), 5)

# Sample 2 files per selected folder
sample_files = []
files_per_folder = 2

for folder in selected_folders:
    files = files_by_folder[folder]
    if len(files) <= files_per_folder:
        sample_files.extend(files)  # take all if fewer
    else:
        sample_files.extend(random.sample(files, files_per_folder))

print(f"📂 Selected folders: {selected_folders}")
print(f"🎯 Total sampled files: {len(sample_files)}")

# Transcription + save to JSON
output_dir = "/content/transcriptions_json"
os.makedirs(output_dir, exist_ok=True)

for path in sample_files:
    print(f"🔊 Transcribing: {path}")
    result = model.transcribe(path)

    # Create a dictionary for JSON output
    output_data = {
        "file_name": os.path.basename(path),
        "language": result["language"],
        "text": result["text"],
        "segments": [
            {
                "start": segment["start"],
                "end": segment["end"],
                "text": segment["text"]
            }
            for segment in result["segments"]
        ]
    }

    # Save to JSON
    json_name = os.path.basename(path).replace(".wav", ".json")
    with open(os.path.join(output_dir, json_name), 'w') as f:
        json.dump(output_data, f, indent=2)

    print(f"✅ Saved to {json_name}")


📂 Selected folders: ['118436', '135031', '284437', '4943', '24615']
🎯 Total sampled files: 10
🔊 Transcribing: /content/audio/dev-clean/251/118436/251_118436_000022_000001.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ Saved to 251_118436_000022_000001.json
🔊 Transcribing: /content/audio/dev-clean/251/118436/251_118436_000021_000000.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ Saved to 251_118436_000021_000000.json
🔊 Transcribing: /content/audio/dev-clean/1272/135031/1272_135031_000052_000001.wav


...

✅ Saved to 3752_4943_000058_000000.json
🔊 Transcribing: /content/audio/dev-clean/5338/24615/5338_24615_000002_000002.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ Saved to 5338_24615_000002_000002.json
🔊 Transcribing: /content/audio/dev-clean/5338/24615/5338_24615_000004_000002.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ Saved to 5338_24615_000004_000002.json


In [96]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.6 MB/s eta 0:00:00


## Transcription + Speaker Diarization

In [61]:
!pip install git+https://github.com/pyannote/pyannote-audio

  Cloning https://github.com/pyannote/pyannote-audio to /tmp/pip-req-build-wz_j59wi
  Running command git clone --filter=blob:none --quiet https://github.com/pyannote/pyannote-audio /tmp/pip-req-build-wz_j59wi
  Resolved https://github.com/pyannote/pyannote-audio to commit 240a7f3ef60bc613169df860b536b10e338dbf3c
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done


In [67]:
from huggingface_hub import login

login()  # will prompt for your Hugging Face token


In [71]:
# from huggingface_hub import whoami
# print(whoami())


{'type': 'user', 'id': '67c7ef27b1a692388e93e7d1', 'name': 'J1asleen', 'fullname': 'Kaur', 'email': 'jasstronger@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/dd50bca5aad41e9c678f9d5ca53b131b.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'token_speechtotext', 'role': 'read', 'createdAt': '2025-06-04T12:49:39.398Z'}}}


In [73]:
from pyannote.audio import Pipeline

pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=True)


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/hyperparams.yaml'
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cu124. Bad things might happen unless you revert torch to 1.x.


DEBUG:speechbrain.utils.parameter_transfer:Collecting files (or symlinks) for pretraining in /root/.cache/torch/pyannote/speechbrain.
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/embedding_model.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["embedding_model"] = /root/.cache/torch/pyannote/speechbrain/embedding_model.ckpt
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/mean_var_norm_emb.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["mean_var_norm_emb"] = /root/.cache/torch/pyannote/speechbrain/mean_var_norm_emb.ckpt
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/classifier.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["classifier"] = /root/.cache/torch/pyannote/speech

In [91]:
import shutil
import os

folder_path = "/content/diarized_outputs"

# Check if folder exists
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)  # Delete the entire folder and its contents
    print(f"✅ Cleared: {folder_path}")
    os.makedirs(folder_path, exist_ok=True)  # Recreate empty folder
    print(f"📁 Recreated: {folder_path}")
else:
    print(f"⚠️ Folder not found: {folder_path}")


✅ Cleared: /content/diarized_outputs
📁 Recreated: /content/diarized_outputs


In [92]:
import os
import json
from pyannote.audio import Pipeline

# Folder paths
transcription_folder = "/content/transcriptions_json"
audio_base_folder = "/content/audio/dev-clean"
output_folder = "/content/diarized_outputs"

os.makedirs(output_folder, exist_ok=True)

# Load diarization pipeline once (make sure your Hugging Face token is configured)
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=True)

# Iterate over all transcription json files
for json_file in os.listdir(transcription_folder):
    if not json_file.endswith(".json"):
        continue

    json_path = os.path.join(transcription_folder, json_file)
    print(f"Processing {json_path} ...")

    # Load transcription result
    with open(json_path, "r") as f:
        result = json.load(f)

    # Derive audio file path from transcription filename
    # Assuming transcription filename: 1993_147965_000003_000003.json
    # and audio path: /content/audio/dev-clean/1993/147965/1993_147965_000003_000003.wav
    name_parts = json_file[:-5].split('_')
    if len(name_parts) < 2:
        print(f"Filename format unexpected: {json_file}")
        continue

    # Construct audio path dynamically from parts (adjust if your folder structure differs)
    year_folder = name_parts[0]
    id_folder = name_parts[1]
    audio_file_path = os.path.join(audio_base_folder, year_folder, id_folder, json_file[:-5] + ".wav")

    if not os.path.isfile(audio_file_path):
        print(f"Audio file not found: {audio_file_path}")
        continue

    # Run diarization
    diarization = pipeline(audio_file_path)

    segments = []
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        segment_text = ""
        for s in result['segments']:
            if s['end'] > turn.start and s['start'] < turn.end:
                segment_text += s['text'] + " "
        segments.append({
            "start": round(turn.start, 2),
            "end": round(turn.end, 2),
            "speaker": speaker,
            "text": segment_text.strip()
        })

    output_data = {
        "file": result.get("file_name", os.path.basename(audio_file_path)),
        "language": result.get("language", "unknown"),
        "segments": segments
    }

    # Save diarized output JSON
    output_file = os.path.join(output_folder, json_file)
    with open(output_file, "w") as f:
        json.dump(output_data, f, indent=2)

    print(f"✅ Saved diarized output: {output_file}")

print("✅ All files processed.")


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/hyperparams.yaml'
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cu124. Bad things might happen unless you revert torch to 1.x.


DEBUG:speechbrain.utils.parameter_transfer:Collecting files (or symlinks) for pretraining in /root/.cache/torch/pyannote/speechbrain.
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/embedding_model.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["embedding_model"] = /root/.cache/torch/pyannote/speechbrain/embedding_model.ckpt
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/mean_var_norm_emb.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["mean_var_norm_emb"] = /root/.cache/torch/pyannote/speechbrain/mean_var_norm_emb.ckpt
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/classifier.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["classifier"] = /root/.cache/torch/pyannote/speech

Processing /content/transcriptions_json/251_118436_000021_000000.json ...
✅ Saved diarized output: /content/diarized_outputs/251_118436_000021_000000.json
Processing /content/transcriptions_json/5338_284437_000031_000001.json ...
✅ Saved diarized output: /content/diarized_outputs/5338_284437_000031_000001.json
Processing /content/transcriptions_json/5338_284437_000013_000001.json ...
✅ Saved diarized output: /content/diarized_outputs/5338_284437_000013_000001.json
Processing /content/transcriptions_json/3752_4943_000022_000002.json ...
✅ Saved diarized output: /content/diarized_outputs/3752_4943_000022_000002.json
Processing /content/transcriptions_json/1272_135031_000030_000000.json ...
✅ Saved diarized output: /content/diarized_outputs/1272_135031_000030_000000.json
Processing /content/transcriptions_json/5338_24615_000002_000002.json ...
✅ Saved diarized output: /content/diarized_outputs/5338_24615_000002_000002.json
Processing /content/transcriptions_json/5338_24615_000004_000002.j

In [93]:
speakers = set()
for _, _, speaker in diarization.itertracks(yield_label=True):
    speakers.add(speaker)
print("Speakers detected:", speakers)


Speakers detected: {'SPEAKER_00'}
